# 오차역전파법을 적용한 신경망 구현하기 

### 신경망 학습의 전체 그림 
![](./images/fig_NN_learning_process.png)

2층 신경망을 구현할 거임 <br/>
> TwoLayerNet 클래스를 정의해서 구현함

### TwoLayerNet 클래스의 인스턴스 변수 
![](./images/fig_TwoLayerNet.png)

### TwoLayerNet 클래스의 메소드(= 함수)
![](./images/fig_TwoLayerNet_method2.png)

###  "4.5 학습 알고리즘 구현하기" 와 다른 점 (p. 136)
이번 구현에서는 계층(Affine, activation, last layer)을 사용함 
> 계층을 사용했기 때문에 단 두가지의 전파만 사용함 <br>
  * 인식 결과를 얻는 처리(predict()) 계층의 전파 <br>
  * 기울기를 구하는(gradient()) 계층의 전파 

In [1]:
# coding: utf-8
import sys, os
sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 설정

In [2]:
import numpy as np
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict

In [3]:
class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
        
        # 매개변수 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size) 
        self.params['b2'] = np.zeros(output_size)

        # 계층 생성
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])

        self.lastLayer = SoftmaxWithLoss()
       
    # 예측 
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        
        return x
        
    # x : 입력 데이터, t : 정답 레이블
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
        
    # x : 입력 데이터, t : 정답 레이블
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads
        
    def gradient(self, x, t):
        
        # forward 순전파 
        self.loss(x, t)

        # backward 역전파 
        dout = 1
        dout = self.lastLayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W2'], grads['b2'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads

### 집중해서 볼 곳 
(1) 계층 생성 블럭 <br/> 
(2) 예측 블럭  <br/>
(3) 역전파 블럭 

특히 계층 생성 블럭에서 <span style="color:red">self</span>.layers를 <span style="color:blue">OrderedDict()</span> 클래스로 인스턴스화 하는것에 주목!

OrderedDict()는 순서가 있는 딕셔너리임. <br/> 
> 순서를 기억하는 사전형 클래스 


__'순서가 있는'__이기 때문에 딕셔너리에 추가된 순서대로 기억함
> 그래서 순전파 때, 추가한 순서대로 각 계층의 forward() 메소드를 호출하기만 하면 끝  <br/>

> 역전파 때는 계층을 역순으로 호출만 하면 됨 

Affine 계층과 ReLU 계층이 각자의 내부에서 순전파와 역전파를 제대로 처리하고 있으니, <br/>
* 여기서는 그냥 계층들을 올바른 순서로 연결한 다음 ( Affine &rarr; ReLu &rarr; Affine &rarr; Softmax)
* 순서대로(혹은 역순으로) 호출하면 끝 

### 신경망 구성 요소를 계층 블록으로 구현하는 장점 
* 쉽게 신경망을 구축함  <br/>
* '계층'으로 모듈화해서 구현한 효과 
> 5층, 10층, 20층, ... 같은 <span style="color:red">깊은 신경망<sup>Deep-NN</sup>을</span> 쉽게 만듬
   * 단순히 필요한 만큼 계층을 추가하면 되니까(레고 블록 조립) 


## dict() vs OrderedDict() 클래스 비교 

In [4]:
list = ( ['a',1], ['b',2], ['c',3], ['d',4] ) 

normalDic = dict(list) 
orderDic = OrderedDict(list)

In [5]:
print("orderDic = ", orderDic)
print("normalDict = ", normalDic)

orderDic =  OrderedDict([('a', 1), ('b', 2), ('c', 3), ('d', 4)])
normalDict =  {'a': 1, 'b': 2, 'c': 3, 'd': 4}


In [6]:
for key, value in normalDic.items():
    print(key, value)

a 1
b 2
c 3
d 4


In [7]:
for key, value in orderDic.items():
    print(key, value)

a 1
b 2
c 3
d 4


ㅎㅎㅎ normalDict 객체가 순서 없이 값을 출력해야하는데... <br/>
Jupyter notebook은 일단 출력은 이렇게 하나봄 <br/>

__참고 사항__ <br/>
* dict() 객체는 내부 자료값만 같으면 순서상관 없이 같다고 판단 <br/>

* OrderedDict() 객체는 개부 자료값 및 순서도 같아야 같다고 판단 

In [8]:
list1 = ( ['a',1], ['b',2], ['c',3], ['d',4] ) 
list2 = ( ['c',3], ['b',2], ['a',1], ['d',4] )

norDic1 = dict(list1)
norDic2 = dict(list2)

orderDic1 = OrderedDict(list1)
orderDic2 = OrderedDict(list2)

In [9]:
print(norDic1 == norDic2)          # 순서 상관 없음 

True


In [10]:
print(orderDic1 == orderDic2)      # 순서가 달라서 False 출력 

False
